# Chain-of-Thought as Redundant Channel Coding

Chain-of-Thought (CoT) prompts encourage LLMs to emit intermediate reasoning steps. This notebook treats those steps as redundancy: extra symbols that can detect or correct semantic errors. We compare a direct transmission of an 8-bit answer through a binary symmetric channel (BSC) with a simple CoT-style scheme that repeats each bit three times (majority vote).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")
np.random.seed(0)



In [ ]:
def transmit(bits, flip_prob, redundancy=1):
    bits = np.array(bits)
    repeated = np.repeat(bits, redundancy)
    noise = np.random.rand(len(repeated)) < flip_prob
    received = np.logical_xor(repeated, noise).astype(int)
    if redundancy == 1:
        decoded = received
    else:
        decoded = received.reshape(-1, redundancy).sum(axis=1) >= (redundancy // 2 + 1)
        decoded = decoded.astype(int)
    return decoded


def simulate(flip_prob, trials=2000):
    direct_errors = 0
    cot_errors = 0
    for _ in range(trials):
        bits = np.random.randint(0, 2, size=8)
        decoded_direct = transmit(bits, flip_prob, redundancy=1)
        decoded_cot = transmit(bits, flip_prob, redundancy=3)
        direct_errors += int(np.any(decoded_direct != bits))
        cot_errors += int(np.any(decoded_cot != bits))
    return direct_errors / trials, cot_errors / trials

flip_probs = np.linspace(0.01, 0.3, 15)
results = np.array([simulate(p) for p in flip_probs])



In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(flip_probs, results[:, 0], marker="o", label="Direct decoding")
ax.plot(flip_probs, results[:, 1], marker="s", label="CoT-style redundancy")
ax.set_xlabel("Bit flip probability (channel noise)")
ax.set_ylabel("Answer error rate")
ax.set_title("Chain-of-Thought as error-control coding")
ax.legend()
plt.show()



Repeating each bit three times (analogous to emitting intermediate reasoning steps) dramatically lowers the probability that the final answer drifts, especially when channel noise exceeds 10%. In practice, CoT adds structured redundancy instead of literal bit repetition, but the effect—stabilizing trajectories via semantic parity checks—is the same.
